## P0 Test Suite
#### Original Author: Emil Sekerinski, McMaster University, February 2017
#### Modified by Dom DiPasquale, Patrick Jakubowski, and Mustafa Haddara

In [ ]:
import os
import nbimporter
from P0 import compileString
from ST import printSymTab

The following method compiles the provided source both with and without optimizations and renders the results nicely for comparison.

In [ ]:
def compareOutput(src):
    compileString(src, 'reg', target='mips')
    compileString(src, 'opt', target='opt')
    
    with open('reg') as f, open('opt') as o:
        reg = ['Orginal Compiler'] + f.read().split('\n')
        opt = ['Optmized Compiler'] + o.read().split('\n')
    os.remove('reg')
    os.remove('opt')
    
    tabsize = 8
    def rmtabs(s):
        res = ''
        for i in range(len(s)):
            if s[i] == '\t':
                res = res + ' ' * (tabsize-(i%tabsize))
            else:
                res = res + s[i]
        return res

    maxlinelen = 30
    def rightpad(s, n=maxlinelen, c=' '):
        clean = rmtabs(s)
        while len(clean) < n:
            clean = clean + c
        return clean
    
    for i in range(len(reg)):
        ls = rightpad(reg[i]) + '| '
        rs = opt[i] if i < len(opt) else ''
        print(ls+rs)

## TODO Dom fill in unused procedures

The following example demonstrates dead store elimination. Notice that although we store the value `5` to variable `b`, we never read that value from that variable. Thus the `sw` operation is unneeded.

Future work includes eliminating unused writes to registers, which would save instructions. However, since these writes are typically `add` or `addi` operations, the only savings we will see will be from fewer operations; `add` and `addi` operations are typically very quick instructions, especially compared to `sw` operations, which can take far longer.

## TODO fix this bug (we should allocate memory for `a`)

In [ ]:
compareOutput("""program p;
    var a,b: integer;
    begin
        b := 4;
        a := 10 + b;
        b := 5;
        write(a)
    end
    """)

The following example demonstrates unused variable elimination. Not only do we not allocate space for variable `c`, we also perform the dead store elimination first, so we are able to detect that variable `b` is never used, so we are allowed to remove that memory declaration as well.

In [ ]:
compareOutput("""program p;
    var a,b,c: integer;
    begin
        b := 4;
        a := 10;
        b := 5;
        write(a)
    end
    """)

## TODO Pat fill in CSE

The following examples demonstrate all of the optimizations working in conjunction.

In [ ]:
compareOutput("""program p;
      type T = record d,e,f:integer end;
      type A = array[0..9] of integer;
      var c: integer;
      var b: boolean;
      var w: A;
      procedure q(var z: integer);
            type T = record d,e:integer end;
            type A = array[0..9] of integer;
            var y: integer;
            var m,n: boolean;
            var w: T;
            var wd, we: integer;

            begin b:=true; w.d := 9; w.e := 5; w.d := 3 - 1 end;
      begin 
      w[1] := 9; w[2] := 5;
        c := 10;
        c := c + 1;
        write(c);
        q(c)
      end
    """)

In [ ]:
compareOutput("""program p;
  var g: integer;          {global variable}
  procedure q(v: integer); {value parameter}
    var l: integer;        {local variable}
    procedure qx(v: integer); {value parameter}
      var l: integer;        {local variable}
      procedure qxt(v: integer); {value parameter}
        var l: integer;        {local variable}
        begin
          l := 9
        end;
      begin
        l := 9;
        q(l)
      end;
    procedure qc(v: integer); {value parameter}
      var l: integer;        {local variable}
      begin
        l := 9;
        q(l)
      end;
    begin
      l := 9;
      qx(l);
      if l > v then
         write(l)
      else
         write(g)
    end;
  procedure x(v: integer); {value parameter}
    var l: integer;        {local variable}
    begin
      l := 9;
      q(l)
    end;
  begin
    g := 5;
    q(7)
  end
""")

In [ ]:
compareOutput("""program p;
  var g: integer;          {global variable}
  procedure x(v: integer); {value parameter}
    var l: integer;        {local variable}
    procedure x2(v: integer); {value parameter}
    var l: integer;        {local variable}
    begin
      l := 9;
      x2(l)
    end;
    begin
      l := 9;
      x2(l)
    end;
  begin
    g := 5;
    x(g)
  end
""")